# Scripts to do QC on the rift detectors

## Load saved pickle of atl06 data

In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

dataset_path = '/data/fast0/datasets/'
atl06_path = "/data/fast1/arc/atl06"
output_path = '/data/fast1/arc/rift_obs'


In [2]:
shelf_name = 'brunt'

atl06_file_name = os.path.join(atl06_path, shelf_name + '.pkl')

with open(atl06_file_name, 'rb') as handle:
    atl06_data = pickle.load(handle)
atl06_data = pd.DataFrame(atl06_data)

rift_obs_output_file_name = os.path.join(output_path, shelf_name + '.pkl')
with open(rift_obs_output_file_name, 'rb') as handle:
    rift_obs = pickle.load(handle)

## Plot some observations

In [3]:
unique_rgts = rift_obs.rgt.unique()

plt.subplots(figsize=(15,5))
for i, row in atl06_data.iterrows():
    these_rifts = rift_obs[rift_obs.data_row==i]
    if len(these_rifts)==0:
        continue
#     print(these_rifts)
    d = np.sqrt( row.x**2 + row.y**2 )
    plt.scatter(d,row.h)
    dr = np.sqrt( these_rifts['x-centroid']**2 + these_rifts['y-centroid']**2 )
    plt.scatter(dr,np.zeros_like(dr),c='r')
    plt.ylim([-10,50])
    break